# 載入套件

In [1]:
#!pip install tensorflow
#!pip install xgboost

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import tensorflow
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score,confusion_matrix

In [2]:
pd.set_option('display.max_columns', None)
df=pd.read_csv('0416_機器學習.csv')
from sklearn.utils import shuffle
df=shuffle(df)


In [3]:
df=df.drop('department_name',axis=1) #沒刪乾淨 之後確定
df=df.drop('fashion_news_frequency',axis=1)
df=df.drop('FN',axis=1)
df=df.drop('Active',axis=1)
df=df.drop('club_member_status',axis=1)

# 開始訓練

In [4]:
from sklearn.preprocessing import LabelEncoder #需要特徵都轉數值
labelencoder = LabelEncoder()

df['product_group_name']= labelencoder.fit_transform(df['product_group_name'])
df['graphical_appearance_name']= labelencoder.fit_transform(df['graphical_appearance_name'])
df['colour_group_name']= labelencoder.fit_transform(df['colour_group_name'])
df['section_name']= labelencoder.fit_transform(df['section_name'])
df['garment_group_name']= labelencoder.fit_transform(df['garment_group_name'])
#以下實驗
df['Year']= labelencoder.fit_transform(df['Year'])
df['pricelabel']= labelencoder.fit_transform(df['pricelabel'])
df['season']= labelencoder.fit_transform(df['season'])
df['salelabel']= labelencoder.fit_transform(df['salelabel'])

df.head()

,article_id,sales_channel_id,product_group_name,graphical_appearance_name,colour_group_name,section_name,garment_group_name,Year,season,pricelabel,salelabel
29164,637549001,1,3,12,1,20,6,0,0,3,0
58693,542464004,2,7,0,0,26,14,1,3,2,0
69554,733029001,2,3,8,1,20,5,1,1,1,0
45284,723151001,2,3,20,20,5,1,1,1,1,0
11370,804992018,1,3,12,35,11,5,2,3,2,0


In [5]:
sdf=df

In [6]:
#類型轉換瘦身 使機器學習運算更快
sdf['garment_group_name']=sdf['garment_group_name'].astype('int32')
sdf['section_name']=sdf['section_name'].astype('int32')
sdf['colour_group_name']=sdf['colour_group_name'].astype('int32')
sdf['graphical_appearance_name']=sdf['graphical_appearance_name'].astype('int32')
sdf['product_group_name']=sdf['product_group_name'].astype('int32')
sdf['sales_channel_id']=sdf['sales_channel_id'].astype('int32')
sdf['article_id']=sdf['article_id'].astype('int32')

sdf['Year']=sdf['Year'].astype('int32')
sdf['season']=sdf['season'].astype('int32')
sdf['pricelabel']=sdf['pricelabel'].astype('int32')
sdf['salelabel']=sdf['salelabel'].astype('int32')

sdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95084 entries, 29164 to 16126
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   article_id                 95084 non-null  int32
 1   sales_channel_id           95084 non-null  int32
 2   product_group_name         95084 non-null  int32
 3   graphical_appearance_name  95084 non-null  int32
 4   colour_group_name          95084 non-null  int32
 5   section_name               95084 non-null  int32
 6   garment_group_name         95084 non-null  int32
 7   Year                       95084 non-null  int32
 8   season                     95084 non-null  int32
 9   pricelabel                 95084 non-null  int32
 10  salelabel                  95084 non-null  int32
dtypes: int32(11)
memory usage: 4.7 MB


In [7]:
#設定要預測的Y
labels=sdf['article_id'] 

In [8]:
#移除要預測的Y 表就只剩下X
sdf=sdf.drop('article_id',axis=1) 

In [9]:
#訓練集、測試集分割
import tensorflow as tf 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sdf, labels,test_size=0.2, random_state=80) 

# xgboost

In [20]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier(        
        n_estimators=8,     #樹的個數
        learning_rate= 0.1,    # 如同學習率 
        max_depth=15,       # 構建樹的深度，越大越容易過擬合    
        subsample=1,         # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0,         # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=1,          # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0,        #最大增量步長，我們允許每個樹的權重估計。
        colsample_bytree=1,         # 生成樹時進行的列取樣 
        min_child_weight=1, 
                     # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
                     # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
                     #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        seed=1000         #隨機種子
#        reg_alpha=0,        # L1 正則項引數
        #scale_pos_weight=1,        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #objective= 'multi:softmax',         #多分類的問題 指定學習任務和相應的學習目標
        #num_class=10,        # 類別數，多分類與 multisoftmax 並用
#        silent=0 ,        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        nthread=4,        # cpu 執行緒數 預設最大
        #eval_metric= 'auc'
)

model.fit(X_train, y_train) 



C:\Users\a0932\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:12:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=15, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=8, n_jobs=6,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=1000, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=None, seed=1000, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:

import pickle
# Fit the model on training set
# save the model to disk

# 將訓練模型轉pickle 
MODEL_x2article = 'MODEL_x2article.sav'
#pickle.dump(xgbc, open(MODEL_x2article, 'wb'))

pickle.dump(model, open('MODEL_x2article.pkl','wb'))

pickle.dump(model, open('MODEL_x2article.json','wb'))

# 從這開始 由視覺化網站的python 檔執行


In [12]:
# 載入機器學習已經訓練好的模型
loaded_model = pickle.load(open(MODEL_x2article, 'rb'))
XGB_pred=loaded_model.predict(X_test) #模型去預測X_test
print( "網頁需要輸入的X:",X_test)

print("網頁預測結果:",XGB_pred) #此為根據X_test預測出的Y

網頁需要輸入的X:        sales_channel_id  product_group_name  graphical_appearance_name  \
71037                 2                   7                         20   
8942                  2                   7                          0   
87956                 1                   2                         19   
87434                 2                   1                         20   
7912                  2                   7                          0   
...                 ...                 ...                        ...   
53057                 2                   6                         19   
77140                 2                   7                         19   
13935                 2                   7                         19   
29931                 1                   8                         11   
56890                 2                   1                          0   

       colour_group_name  section_name  garment_group_name  Year  season  \
71037                 20 

In [13]:
XGBtrain_score=xgbc.score(X_train, y_train)
print(XGBtrain_score) #X與Y訓練集的準確率

0.817187479458898


In [14]:
# XGBtest_score=xgbc.score(X_test,y_test)
XGBtest_score = loaded_model.score(X_test, y_test)

print('The Accuracy of XGBOOST Classifier on testing set:', XGBtest_score) #X與Y測試集的準確率

The Accuracy of XGBOOST Classifier on testing set: 0.7970763001524951


In [15]:
# 達到一定標準證明模型準確可以使用
# 只要再製造X_test 就可以print(XGB_pred)得到要的Y

In [16]:
#建立X_test架構====想法
#填入網頁輸入內容連接的函式?

請輸入購買管道=pd.Series('線上') #此為透過網頁輸入的資料
請輸入想要類別=pd.Series('上衣') #此為透過網頁輸入的資料
請輸入想要樣式=pd.Series('圓形') #此為透過網頁輸入的資料

TestX=pd.DataFrame({'sales_channel_id':請輸入購買管道,
                   'product_group_name':請輸入想要類別,
                   'graphical_appearance_name':請輸入想要樣式})
TestX

,sales_channel_id,product_group_name,graphical_appearance_name
0,線上,上衣,圓形


In [17]:
#以上方想法請參考一條龍的檔案